# Running graphs with `tf.Session()`

The `tf.Session` class is used to launch a graph that has been previously created. In this example, our computational graph implements just a counter:



In [1]:
import tensorflow as tf
tf.reset_default_graph()

state = tf.Variable(0, name='state')
step = tf.constant(1, name='step')
increment = tf.add(state, step, name='increment')
update = tf.assign(state, increment, name='update')

Now, let's try to run our counter in a session.  
First, we create a new session object.

In [2]:
sess = tf.Session()
print(sess)

At the beginning of a session lifecycle, before running graph, you need to explicitly initialize all the variables to be used in the session. 

In [3]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

Then we can run our computation.

In [4]:
print sess.run(state)
for _ in range(9):
    print sess.run(update)

0
1
2
3
4
5
6
7
8
9


A `tf.Session` can owns resources that must be released at the end of the session lifecycle.

In [5]:
sess.close()

Ok, let's put all together and use the pythonic context idiom so that we shall not care about disposing the sessio  object and releasing resources:

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(state)
    for i in range(9):
        print sess.run(update)

0
1
2
3
4
5
6
7
8
9


...et voilà!

## Saving and restoring

You might want to save the state of your model during the training. TF offers you a very easy way to persist the state of your session.  
  
First, let's build a model to compute the `AND` logical function between two input signals.

In [7]:
tf.reset_default_graph()

xy = tf.placeholder(name='xy', dtype=tf.float32, shape=[None, 2])
x_and_y = tf.placeholder(name='x_and_y', dtype=tf.float32, shape=[None, 1])
w = tf.get_variable(name='w', dtype=tf.float32, shape=[2, 1])
b = tf.get_variable(name='b', dtype=tf.float32, shape=[])
logits = tf.matmul(xy, w) + b
pred = tf.sigmoid(logits)
loss_op = tf.losses.mean_squared_error(x_and_y, pred)
train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss_op)

The `get_data()` function returns batches of 32 examples.

In [8]:
import numpy
import random

def get_data():
    xs = [random.randint(0, 1) for _ in range(32)]
    ys = [random.randint(0, 1) for _ in range(32)]
    xy = numpy.asarray([[x, y] for x, y in zip(xs, ys)], dtype=numpy.float32)
    x_and_y = numpy.asarray([[x * y] for x, y in zip(xs, ys)], dtype=numpy.float32)
    return xy, x_and_y

Then, we run training for 100 steps, running a single evaluation step every 10 steps and saving the current state of the session. To save it, we have to create a `tf.Saver` objects and set a pysical path that will be used to persist the session state in the so called *model checkpoint*.

In [9]:
import os
import shutil

CHECKPOINT_DIR = '/tmp/TF-102-03/'
CHECKPOINT_NAME = 'MODEL'
if os.path.isdir(CHECKPOINT_DIR):
    shutil.rmtree(CHECKPOINT_DIR)
os.mkdir(CHECKPOINT_DIR)
CHECKPOINT = os.path.join(CHECKPOINT_DIR, CHECKPOINT_NAME)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1000):
        inp, target = get_data()
        sess.run(train_op, feed_dict={
                xy: inp,
                x_and_y: target
            })
        
        # Evaluation!
        if step % 100 == 0:
            loss = sess.run(loss_op, feed_dict={
                xy: inp,
                x_and_y: target
            })
            checkpoint = saver.save(sess, CHECKPOINT, global_step=step)
            print('step %d: loss %f -- stored in %s' % (step, loss, checkpoint))

step 0: loss 0.278275 -- stored in /tmp/TF-102-03/MODEL-0
step 100: loss 0.186331 -- stored in /tmp/TF-102-03/MODEL-100
step 200: loss 0.113936 -- stored in /tmp/TF-102-03/MODEL-200
step 300: loss 0.111630 -- stored in /tmp/TF-102-03/MODEL-300
step 400: loss 0.095649 -- stored in /tmp/TF-102-03/MODEL-400
step 500: loss 0.090721 -- stored in /tmp/TF-102-03/MODEL-500
step 600: loss 0.080667 -- stored in /tmp/TF-102-03/MODEL-600
step 700: loss 0.068780 -- stored in /tmp/TF-102-03/MODEL-700
step 800: loss 0.060936 -- stored in /tmp/TF-102-03/MODEL-800
step 900: loss 0.062061 -- stored in /tmp/TF-102-03/MODEL-900


We can reload the restore the latest session and reuse the model: the `loss` values will be comparable with the latest ones.

In [24]:
saver = tf.train.Saver()
with tf.Session() as sess:
    checkpoint = tf.train.latest_checkpoint(CHECKPOINT_DIR)
    saver.restore(sess, checkpoint)
    for _ in range(5):
        loss = sess.run(loss_op, feed_dict={
                    xy: inp,
                    x_and_y: target
                })
        print('loss: %f' % loss)

INFO:tensorflow:Restoring parameters from /tmp/TF-102-03/MODEL-900


INFO:tensorflow:Restoring parameters from /tmp/TF-102-03/MODEL-900


loss: 0.053307
loss: 0.053307
loss: 0.053307
loss: 0.053307
loss: 0.053307
